In [ ]:
import numpy as np
import cv2
from network_full import *
import os

fer2013_label={'angry':0,'disgust':1,'fear':2,'happy':3,'neutral':4,'sad':5,'surprise':6}

In [8]:
#data loading
def load_data_fer(dataset,path):
    labels= os.listdir(os.path.join(path,dataset))
    X=[]
    Y=[]

    for label in labels:
        for file in os.listdir(os.path.join(path,dataset,label)):
            image=cv2.imread(os.path.join(path,dataset,label,file),cv2.IMREAD_UNCHANGED)

            X.append(image)
            Y.append(fer2013_label[label])
    return np.array(X) , np.array(Y).astype('uint8')

def create_data_fer(path):
    X,Y=load_data_fer('train',path)
    x_train,y_train=load_data_fer('test',path)
    return X,Y,x_train,y_train

x,y,x_test,y_test= create_data_fer('FER2013')
x[0].shape

(48, 48)

In [9]:
#flattening & normalisation
x=(x.reshape(x.shape[0],x.shape[1]*x.shape[2]).astype(np.float32)-127.5)/127.5
x_test=(x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]).astype(np.float32)-127.5)/127.5


In [10]:
#shuffle data 
keys= np.array(range(x.shape[0]))
np.random.shuffle(keys)
x=x[keys]
y=y[keys]
y[:5]

array([3, 3, 2, 5, 2], dtype=uint8)

In [11]:
#model build up
model=Model()
model.add(layer_dense(x.shape[1],512))
model.add(Activation_ReLu())
model.add(Layer_Dropout(0.25))
model.add(layer_dense(512,256))
model.add(Activation_ReLu())
model.add(Layer_Dropout(0.25))
model.add(layer_dense(256,128))
model.add(Activation_ReLu())
model.add(Layer_Dropout(0.25))
model.add(layer_dense(128,7))
model.add(Activation_SoftMax())
model.set(loss=Loss_CategoricalCrossEntropy(),
          accuracy=Accuracy_Categorical(),
          optimizer=Optimizer_Adam(learning_rate=0.001,decay=5e-5))
model.finalize()


In [12]:
#training
model.train(x,y,epochs=20,validation_data=(x_test,y_test),batch_size=128)


epoch: 1
 step: 0, acc: 0.141, loss: 1.946,data_loss: 1.946, reg_loss: 0.000 lr: 0.001000
 step: 1, acc: 0.172, loss: 1.945,data_loss: 1.945, reg_loss: 0.000 lr: 0.001000
 step: 2, acc: 0.141, loss: 1.943,data_loss: 1.943, reg_loss: 0.000 lr: 0.001000
 step: 3, acc: 0.203, loss: 1.937,data_loss: 1.937, reg_loss: 0.000 lr: 0.001000
 step: 4, acc: 0.258, loss: 1.924,data_loss: 1.924, reg_loss: 0.000 lr: 0.001000
 step: 5, acc: 0.250, loss: 1.904,data_loss: 1.904, reg_loss: 0.000 lr: 0.001000
 step: 6, acc: 0.289, loss: 1.860,data_loss: 1.860, reg_loss: 0.000 lr: 0.001000
 step: 7, acc: 0.172, loss: 1.899,data_loss: 1.899, reg_loss: 0.000 lr: 0.001000
 step: 8, acc: 0.273, loss: 1.814,data_loss: 1.814, reg_loss: 0.000 lr: 0.001000
 step: 9, acc: 0.266, loss: 1.832,data_loss: 1.832, reg_loss: 0.000 lr: 0.001000
 step: 10, acc: 0.344, loss: 1.825,data_loss: 1.825, reg_loss: 0.000 lr: 0.001000
 step: 11, acc: 0.320, loss: 1.765,data_loss: 1.765, reg_loss: 0.000 lr: 0.000999
 step: 12, acc: 0

In [ ]:
#loss & accuracy(for both training & validation) visualisation during training 
import matplotlib.pyplot as plt
from network_full import *
L=range(1,21)#number of epochs
plt.figure(1)
plt.plot(L,model.history['accuracy_train'],'b-', label="training accuracy")
plt.plot(L,model.history['accuracy_val'],'g',label="validation accuracy")
plt.title("Accuracy")
plt.legend()
plt.figure(2)
plt.plot(L,model.history['loss_train'],'b-', label="training loss")
plt.plot(L,model.history['loss_val'],'g',label="validation loss")
plt.title("loss")
plt.legend()
plt.show()

In [23]:
#saving the model
model.save('fer13.model')
